In [1]:
from pymongo import MongoClient
import pandas as pd
import pprint as pprint
import collections
from itertools import izip
import math
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.style.use('ggplot')
import numpy as np

from sklearn.linear_model import LinearRegression

# Fix the trans_score

In [2]:
client = MongoClient()
db = client.proj

## Query Mongo for the data 

In [3]:
cursor_sea = db.listings.find({"address.state": "WA"})
cursor_sf = db.listings.find({"address.state": "CA"}) # returns a generator

In [324]:
#results_sf[0]

In [4]:
# taken from Stackoverflow discussion at:
# http://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [5]:
def rename_columns(df):
    #cols = ['_id', 'pageviewcount_currentmonth', 'pageviewcount_total'] # want to keep these names as they are
    df.columns = [col.replace("editedfacts_", '') for col in df.columns]
    df.columns = [col.replace("address_", '') for col in df.columns]
    df.columns = [col.replace("posting_", '') for col in df.columns]
    return df

In [6]:
def drop_columns(df):
    columns = ['image', 'openhousedates', 'mls', 'lastupdateddate',
             'externalurl', 'agentprofileurl', 'agentname', 'homedetails', 'mls', 
             'homeinfo']
    for column in columns:
        try:
            df = df.drop(column, axis=1)
        except:
            continue
    return df

In [7]:
########## takes the architectrue style and bins the home accordingly #######

def get_home_style(architecture):
    '''
    Input: the ['architecture'] column from a pandas df row. Applied
    using the df.apply() method.
    
    Output: one of architecture categories.
    ''' 
    bins = {'practical': ['Craftsman', 'Bungalow'],
           'modern': ['Loft', 'Contemporary', 'Modern'],
           'romantic': ['Spanish', 'French', 'Colonial', 'Georgian',   ],
           'classic': [],
           'other': []}
   

## Init and clean the DF

In [8]:
results_sea = [document for document in cursor_sea]
results_sf = [document for document in cursor_sf]
print len(results_sea)
print len(results_sf)# use this insead of cast as list because some documents are lost with list(cursor)
flattened_sea = [flatten(result) for result in results_sea]
flattened_sf = [flatten(result) for result in results_sf]
# Init the dataframes

df_sea = pd.DataFrame(flattened_sea, index=range(len(flattened_sea)))
df_sf = pd.DataFrame(flattened_sf, index=range(len(flattened_sf)))
df_sea = rename_columns(df_sea)
df_sf = rename_columns(df_sf)
# Rename the columns to something a little easier to read

815
764


In [9]:

# df_sea = drop_columns(df_sea)
# df_sf = drop_columns(df_sf)
df_sea = df_sea.drop_duplicates('zpid')
df_sf = df_sf.drop_duplicates('zpid')
df_sea = df_sea.drop('editedfacts', axis=1)
# We are not interested in recommending empty lots

df_sf = df_sf[~(df_sf.usecode == 'VacantResidentialLand')]
df_sea = df_sea[~(df_sea.usecode == 'VacantResidentialLand')]
print df_sea.shape
print df_sf.shape

(779, 60)
(629, 60)


## Drop the columns not used for the current recommender
### these may be useful for more advanced feature engineering beyond the MVP

In [136]:
analysis_columns = ['bathrooms', 'bedrooms', 'finishedsqft', 'numrooms', 'yearbuilt',
           'price', 'trans_score', 'walkscore_score']
meta_columns = ['zpid', 'street', 'city', 'state', 'zipcode'] 

sf = df_sf[analysis_columns]
sf_meta = df_sf[meta_columns]
sea = df_sea[analysis_columns]
sea_meta = df_sea[meta_columns]

In [14]:
df_sea.to_csv('seattle_test.csv')
df_sf.to_csv('sanfran_test.csv')

In [123]:
#### START DROPPING COLUMNS!!! ###
df1_sf = df_sf.copy()
dff1_sf = df1_sf.drop(['_id', 'city', 'latitude', 'longitude', 'state', 'street', 
                'zipcode', 'appliances', 'architecture', 'basement', 'coolingsystem',
               'coveredparkingspaces', 'exteriormaterial', 'floorcovering',
               'floornumber', 'heatingsources', 'heatingsystem', 'lotsizesqft', 'numfloors', 'numunits',
               'parkingtype', 'roof', 'rooms', 'view', 'yearupdated', 'elementaryschool', 
               'highschool', 'homedescription', 'images_count', 'images_image', 'links_homedetails',
                'links_homeinfo', 'links_photogallery', 'middleschool', 'neighborhood', 'pageviewcount_currentmonth',
             'pageviewcount_total', 'agentname' ,'agentprofileurl', 'externalurl', 'lastupdateddate',
             'mls', 'openhousedates', 'status', 'schooldistrict', 'usecode', 'type', 'trans_desc', 'trans_summary', 'walkscore_desc',
             'whatownerloves', 'zpid'], axis=1)

df1_sea = df_sea.copy()
df1_sea = df1_sea.drop(['_id', 'city', 'latitude', 'longitude', 'state', 'street', 
                'zipcode', 'appliances', 'architecture', 'basement', 'coolingsystem',
               'coveredparkingspaces', 'exteriormaterial', 'floorcovering',
               'floornumber', 'heatingsources', 'heatingsystem', 'lotsizesqft', 'numfloors', 'numunits',
               'parkingtype', 'roof', 'rooms', 'view', 'yearupdated', 'elementaryschool', 
               'highschool', 'homedescription', 'images_count', 'images_image', 'links_homedetails',
                'links_homeinfo', 'links_photogallery', 'middleschool', 'neighborhood', 'pageviewcount_currentmonth',
             'pageviewcount_total', 'agentname' ,'agentprofileurl', 'externalurl', 'lastupdateddate',
             'mls', 'openhousedates', 'status', 'schooldistrict', 'usecode', 'type', 'trans_desc', 'trans_summary', 'walkscore_desc',
             'whatownerloves', 'zpid'], axis=1)

In [138]:
#### Do San Fran first ####

df1_sf = sf.dropna(axis=0, how='any')

df1_sf['bathrooms'] = df1_sf['bathrooms'].astype(float)
df1_sf['bedrooms'] = df1_sf['bedrooms'].astype(int)
df1_sf['finishedsqft'] = df1_sf['finishedsqft'].astype(int)
df1_sf['numrooms'] = df1_sf['numrooms'].astype(int)
df1_sf['yearbuilt'] = df1_sf['yearbuilt'].astype(int)
df1_sf['price'] = df1_sf['price'].astype(int)
df1_sf['trans_score'] = df1_sf['trans_score'].astype(int)
df1_sf['walkscore_score'] = df1_sf['walkscore_score'].astype(int)

#### Seattle next ####

df1_sea = sea.dropna(axis=0, how='any')

df1_sea['bathrooms'] = df1_sea['bathrooms'].astype(float)
df1_sea['bedrooms'] = df1_sea['bedrooms'].astype(int)
df1_sea['finishedsqft'] = df1_sea['finishedsqft'].astype(int)
df1_sea['numrooms'] = df1_sea['numrooms'].astype(int)
df1_sea['yearbuilt'] = df1_sea['yearbuilt'].astype(int)
df1_sea['price'] = df1_sea['price'].astype(int)
df1_sea['trans_score'] = df1_sea['trans_score'].astype(int)
df1_sea['walkscore_score'] = df1_sea['walkscore_score'].astype(int)

/Users/michaelhood/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/michaelhood/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/michaelhood/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: h

In [86]:
def normalize_num(x, col_min, col_max):
    '''Normalize everything from 0 to 1 '''
    return float((x - col_min)) / (col_max - col_min)

In [87]:
def normalize_columns(df):
    for column in df.columns:
        try:
            min_val = df[column].min()
            max_val = df[column].max()
            df[column] = df[column].apply(normalize_num, args=(min_val, max_val))
        except:
            continue
    return df

## Apply normalization to the columns

In [139]:
df_sf = normalize_columns(df1_sf)
df_sea = normalize_columns(df1_sea)
print df_sf.shape
print df_sea.shape

(285, 8)
(176, 8)


/Users/michaelhood/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Now write the dataframes to a csv

In [143]:
pwd

u'/Users/michaelhood/Documents/projects/project/data/san_fran'

In [144]:
cd ../seattle/

/Users/michaelhood/Documents/projects/project/data/seattle


In [145]:
sea.to_csv('analysis_sea.csv')
sea_meta.to_csv('meta_sea.csv')

In [141]:
cd ../san_fran/

/Users/michaelhood/Documents/projects/project/data/san_fran


In [142]:
sf.to_csv('analysis_sf.csv')
sf_meta.to_csv('meta_sf.csv')

In [11]:
#df_sf.ix[0:5]

In [350]:
#df['links_photogallery'][df['architecture'] == 'Craftsman']
#df['links_photogallery']

In [304]:
#df.to_csv('trx_sf.csv', separator=",")

In [358]:
#### num_rooms distance

def num_bedrooms_dist(number_rooms):
    if number_rooms < 10:
        return math.log(number_rooms, 10)
    return 1

In [433]:
# Fill in nan with the mean year of 1955

y = df_sf['yearbuilt']
y = y.fillna(0).astype(int)
mean_yr_built = y[y != 0].mean()
df_sf['yearbuilt'] = df_sf['yearbuilt'].fillna(mean_yr_built).astype(int)

In [356]:
#df_sf['yearbuilt'].plot(kind='hist', bins=20)

In [407]:
# Computes a distance based on the age of a home and take a param for user_pref on age of home

def age_distance(ref):   # instead of using 1.1 and 0.9 I can use a paramter based on a users pref for older or newer homes
    diff = math.fabs(ref - date)
    if date < ref:
        dist = ( 1.1 * diff / 100.0)
        if dist > 1:
            dist = 1
        return dist
    if date > ref:
        return ( 0.9 * diff / 100.0)
    if diff >= 100:
        return 1

In [336]:
#### Determine the parking_distance metric

def score_parking(parking_type, coveredparkingspaces):
    score = 0
    if "Garage - Attached" in parking_type:
        score += 0.5
    elif "Garage - Detached" in parking_type:
        score += 0.4
    elif "Carport" in parking_type:
        score += 0.3
    elif "Off-street" in parking_type:
        score += 0.2
    elif "On-street" in parking_type:
        score += 0.1
    elif "None" in parking_type:
        score += 0.0
    if coveredparkingspaces >= 5:
        score += 0.5
    else:
        score += coveredparkingspaces * 0.1
    return score

In [340]:
# Apply a unit score to usecode, which is the type of listing

def score_usecode(usecode):
    scores = {'SingleFamily': 1.0,
              'Townhouse': 0.9,
              'Miscellaneous': 0.8,
              'Duplex': 0.7,
              'Cooperative': 0.6,
              'Condominium': 0.5,
              'Apartment': 0.4,
              'Quadruplex': 0.3,
              'MultiFamily2To4': 0.2, 
              'MultiFamily5Plus': 0.1,
              'Mobile': 0}
    if usecode in scores:
        return scores[usecode]
    else:
        return 0.5 # this is just a swag

In [40]:
def impute_bedrooms(sqft, bedrooms):  ### This needs to be fixed one bedroom per thousand feet seems too low
    '''
    Input: number of sqft as an int and bedrooms as a string
    Output: integer, representing the number of bedrooms
    '''
    if bedrooms == -1:
        num_thousands = int(sqft) / 1000
        if num_thousands < 1:
            return 1
        return math.floor(num_thousands)
    return bedrooms

In [41]:
def impute_bathrooms(sqft, bathrooms):
    '''
    Input: number of sqft as an int and bathrooms as a string.
    Ouput: if value is nan, returns a float representing the number of bathroooms + half bathrooms
        else, returns nothing
    '''
    if bathrooms == -1:
        thousands = sqft / 1000  # assume one bathroom for every thousands feet of hosue
        total = sqft / 1000.0
        remainder = total - thousands
        fractional_bathrooms = 0
        if total < 1:
            return float(1)
        if remainder >= 0.5:
            fractional_bathrooms += 0.5 # assume half bathrooms for every 500 ft over thousands
            return thousands + fractional_bathrooms
        return float(thousands)
    else:
        return bathrooms

In [415]:
df_sf['parkingtype'] = df_sf['parkingtype'].fillna("None")
df_sf['coveredparkingspaces'] = df_sf['coveredparkingspaces'].fillna(0)
df_sf['coveredparkingspaces'] = df_sf['coveredparkingspaces'].astype(int)
#df_sf[['parkingtype', 'coveredparkingspaces']]

In [422]:
df_sf[['parkingtype', 'coveredparkingspaces']].ix[31]

parkingtype             Garage - Detached, Off-street
coveredparkingspaces                                0
Name: 31, dtype: object

In [434]:
# clean parking columns
df_sf['parkingtype'] = df_sf['parkingtype'].fillna("None")
df_sf['coveredparkingspaces'] = df_sf['coveredparkingspaces'].fillna(0)
df_sf['coveredparkingspaces'] = df_sf['coveredparkingspaces'].astype(int)

# Make a parking_score colun
df_sf['parking_score'] = df_sf[['parkingtype', 'coveredparkingspaces']].apply(lambda row: score_parking(row[0],
                                                                                                       row[1]),
                                                                                                       axis=1)

# convert usecode to a metric
df_sf['usecode'] = df_sf['usecode'].apply(score_usecode)

# covert year built to a useful metric
yearbuilt_min = df_sf['yearbuilt'].min()
yearbuilt_max = df_sf['yearbuilt'].max()
df_sf['yearbuilt'] = df_sf['yearbuilt'].apply(lambda x: normalize_num(x, yearbuilt_min, yearbuilt_max))

In [430]:
a = df_sf[['finishedsqft', 'basement', 'bedrooms', 'bathrooms', 'numfloors', 'numrooms', 'numunits']].copy()
# Drop row 662 because  it is some dilapidated apartment compelx with tons of bogus data fields
# Drop row 168 because it is a huge outlier
a = a[(a.index != 662) & (a.index != 168)]
a['bedrooms'].ix[37] = 1 ## this listing onyl has 437 squarefoot!!!
a['bathrooms'].ix[114] = 1.5 ## this listing probably does not have 6 bathrooms

sqft_median = a['finishedsqft'][~(a['finishedsqft'].isnull())].median()
sqft_mean = a['finishedsqft'][~(a['finishedsqft'].isnull())].mean()

# impute the missing sqft with the median
a['finishedsqft'] = a['finishedsqft'].fillna(sqft_median).astype(int)

a['bathrooms'] = a['bathrooms'].fillna(-1)
a['bedrooms'] = a['bedrooms'].fillna(-1)

# impute the missing bedrooms and bathrooms with imputation functs
a['bathrooms'] = a.apply(lambda x: impute_bathrooms(x['finishedsqft'], x['bathrooms']), axis=1)
a['bedrooms'] = a.apply(lambda x: impute_bedrooms(x['finishedsqft'], x['bedrooms']), axis=1)

# normalize bathrooms
# a['bathrooms'] = a['bathrooms'].astype(float)
# bathrooms_min = a.bathrooms.min()
# bathrooms_max = a.bathrooms.max()
# a['bathrooms'] = a['bathrooms'].apply(normalize_num, args=(bathrooms_min, bathrooms_max))

# covert bedrooms to an appropriate metric
a['bedrooms'] = a['bedrooms'].astype(int)
a['bedrooms'] = a['bedrooms'].apply(num_bedrooms_dist)

numrooms_mean = a['numrooms'][~(a['numrooms'].isnull())].astype(int).mean()
numrooms_med = a['numrooms'][~(a['numrooms'].isnull())].astype(int).median()

# impute na values with the median

a['numrooms'] = a['numrooms'].fillna(numrooms_med).astype(int)
a['basement'] = a['basement'].fillna('None')
a['basement'] = a['basement'].apply(lambda x: 0 if x=='None' else 1)
a = a.drop(['numfloors', 'numunits'], axis=1)
# make a feature that is the ratio of bathrooms to bedrooms
a['bed_bath'] = a['bathrooms'] / a['bedrooms']
#a['bed_bath'] = a['bed_bath'].apply(normalize
# make a feature that is the average sqft per room in the house
a['sqft_room'] = a['finishedsqft'] / a['numrooms']

a = a.drop(['basement'], axis=1) # I'll incorporate this later
for column in a.columns:
    min_val = a[column].min()
    max_val = a[column].max()
    df_sf[column] = a[column].apply(normalize_num, args=(min_val, max_val))
    
df_sf[['finishedsqft', 'bedrooms', 'bathrooms', 'numrooms', 'bed_bath', 'sqft_room']] = a.copy()

TypeError: unsupported operand type(s) for /: 'unicode' and 'float'

In [394]:
#### START DROPPING COLUMNS!!! ###

df_sf = df_sf.drop(['_id', 'city', 'latitude', 'longitude', 'state', 'street', 
                    'zipcode', 'appliances', 'architecture', 'basement', 'coolingsystem',
                   'coveredparkingspaces', 'exteriormaterial', 'finishedsqft', 'floorcovering',
                   'floornumber', 'heatingsources', 'heatingsystem', 'numfloors', 'numunits',
                   'parkingtype', 'roof', 'rooms', 'view'], axis=1)

In [396]:
df_sf.columns

Index([u'bathrooms', u'bedrooms', u'heatingsources', u'heatingsystem', u'lotsizesqft', u'numfloors', u'numrooms', u'numunits', u'parkingtype', u'roof', u'rooms', u'usecode', u'view', u'yearbuilt', u'yearupdated', u'elementaryschool', u'highschool', u'homedescription', u'images_count', u'images_image', u'links_homedetails', u'links_homeinfo', u'links_photogallery', u'middleschool', u'neighborhood', u'pageviewcount_currentmonth', u'pageviewcount_total', u'agentname', u'agentprofileurl', u'externalurl', u'lastupdateddate', u'mls', u'openhousedates', u'status', u'type', u'price', u'schooldistrict', u'trans_desc', u'trans_score', u'trans_summary', u'walkscore_desc', u'walkscore_score', u'whatownerloves', u'zpid', u'parking_score', u'bed_bath', u'sqft_room'], dtype='object')

In [429]:
df_sf

,_id,city,latitude,longitude,state,street,zipcode,appliances,architecture,basement,...,price,schooldistrict,trans_desc,trans_score,trans_summary,walkscore_desc,walkscore_score,whatownerloves,zpid,parking_score
0,55f842bb3ae7404f74f89baf,Sunnyvale,37.36086,-122.054546,CA,1058 Lois Ave,94087,NaN,NaN,NaN,...,NaN,Fremont Union High,Some Transit,34,"6 nearby routes: 6 bus, 0 rail, 0 other",Car-Dependent,24,NaN,19537840,0.0
1,55f842bb3ae7404f74f89bb1,Sunnyvale,37.385314,-122.04329,CA,994 La Mesa Ter UNIT G,94086,NaN,NaN,NaN,...,NaN,NaN,Some Transit,36,"9 nearby routes: 5 bus, 4 rail, 0 other",Car-Dependent,49,NaN,19521633,0.0
2,55f842bb3ae7404f74f89bb3,Sunnyvale,37.350976,-122.006338,CA,1005 Bryant Way APT L,94087,NaN,NaN,NaN,...,NaN,NaN,Some Transit,43,"4 nearby routes: 4 bus, 0 rail, 0 other",Very Walkable,77,NaN,19617959,0.0
3,55f842bb3ae7404f74f89bb5,Sunnyvale,37.371456,-122.056001,CA,1175 Andover Dr,94087,NaN,NaN,NaN,...,NaN,NaN,Some Transit,43,"4 nearby routes: 4 bus, 0 rail, 0 other",Somewhat Walkable,62,NaN,19537196,0.0
4,55f842bb3ae7404f74f89bb9,Sunnyvale,37.403641,-122.011927,CA,1148 La Rochelle Ter UNIT B,94089,NaN,NaN,NaN,...,NaN,NaN,Some Transit,46,"11 nearby routes: 10 bus, 1 rail, 0 other",Somewhat Walkable,54,NaN,19494162,0.0
6,55f6e8a63ae7404c5edef1e7,Alameda,37.774675,-122.252674,CA,1724 Eagle Ave,94501,NaN,NaN,NaN,...,NaN,NaN,Some Transit,41,"8 nearby routes: 8 bus, 0 rail, 0 other",Somewhat Walkable,61,NaN,24861179,0.0
7,55f6e8a63ae7404c5edef1e8,Alameda,37.783418,-122.283656,CA,2378 Coral Sea St,94501,NaN,NaN,NaN,...,NaN,NaN,Some Transit,45,"8 nearby routes: 7 bus, 0 rail, 1 other",Somewhat Walkable,65,NaN,72700378,0.0
12,55f6e8a63ae7404c5edef1f3,Alameda,37.773898,-122.274062,CA,758 Haight Ave,94501,NaN,Other,NaN,...,849900,NaN,Some Transit,46,"7 nearby routes: 7 bus, 0 rail, 0 other",Very Walkable,87,This home has unbelievable Value:\nBoth Upstai...,24863670,0.2
16,55f6e8a63ae7404c5edef1f8,Alameda,37.783023,-122.268962,CA,12 Courageous Ct # B70,94501,NaN,NaN,NaN,...,NaN,NaN,Some Transit,45,"7 nearby routes: 7 bus, 0 rail, 0 other",Somewhat Walkable,53,NaN,24872185,0.0
22,55f6e8a63ae7404c5edef1ff,Alameda,37.76794,-122.24758,CA,2133 Santa Clara Avenue #104,94501,NaN,NaN,NaN,...,429500,NaN,Some Transit,46,"8 nearby routes: 8 bus, 0 rail, 0 other",Very Walkable,89,NaN,2101267475,0.0
